In [1]:
import pandas as pd
import json
import seaborn as sns; sns.set()
from matplotlib import pyplot as plt

plt.ioff()

pd.set_option("display.max_columns", 1000)
pd.set_option("display.max_rows", 1000)

Đường dẫn đến file Data chứa các thông tin của xe bus.

In [337]:
!find /data/ebms/feed/* -type d

/data/ebms/feed/businfo
/data/ebms/feed/businfo/2020
/data/ebms/feed/businfo/2020/03
/data/ebms/feed/businfo/2020/03/10
/data/ebms/feed/businfo/2020/03/11
/data/ebms/feed/businfo/2020/03/12
/data/ebms/feed/businfo/2020/03/13
/data/ebms/feed/businfo/2020/03/14
/data/ebms/feed/businfo/2020/03/15
/data/ebms/feed/businfo/2020/03/16
/data/ebms/feed/businfo/2020/03/17
/data/ebms/feed/businfo/2020/03/18
/data/ebms/feed/businfo/2020/03/19
/data/ebms/feed/businfo/2020/03/20
/data/ebms/feed/businfo/2020/03/23
/data/ebms/feed/businfo/2020/03/24
/data/ebms/feed/businfo/2020/03/25
/data/ebms/feed/businfo/2020/03/26
/data/ebms/feed/businfo/2020/03/27
/data/ebms/feed/businfo/2020/03/28
/data/ebms/feed/businfo/2020/03/29
/data/ebms/feed/businfo/2020/03/30
/data/ebms/feed/businfo/2020/03/31
/data/ebms/feed/businfo/2020/03/21
/data/ebms/feed/businfo/2020/03/22
/data/ebms/feed/businfo/2020/04
/data/ebms/feed/businfo/2020/04/01
/data/ebms/feed/businfo/2020/04/02
/data/ebms/feed/businfo/2020/04/03
/data/eb

Đường dẫn đến data files của tập *prediction* ghi nhận data log của xe bus

In [130]:
!find /data/ebms/feed/prediction/*

/data/ebms/feed/prediction/2020_03_10.csv
/data/ebms/feed/prediction/2020_03_11.csv
/data/ebms/feed/prediction/2020_03_12.csv
/data/ebms/feed/prediction/2020_03_13.csv
/data/ebms/feed/prediction/2020_03_14.csv
/data/ebms/feed/prediction/2020_03_15.csv
/data/ebms/feed/prediction/2020_03_16.csv
/data/ebms/feed/prediction/2020_03_17.csv
/data/ebms/feed/prediction/2020_03_18.csv
/data/ebms/feed/prediction/2020_03_19.csv
/data/ebms/feed/prediction/2020_03_20.csv
/data/ebms/feed/prediction/2020_03_21.csv
/data/ebms/feed/prediction/2020_03_22.csv
/data/ebms/feed/prediction/2020_03_23.csv
/data/ebms/feed/prediction/2020_03_24.csv
/data/ebms/feed/prediction/2020_03_25.csv
/data/ebms/feed/prediction/2020_03_26.csv
/data/ebms/feed/prediction/2020_03_27.csv
/data/ebms/feed/prediction/2020_03_28.csv
/data/ebms/feed/prediction/2020_03_29.csv
/data/ebms/feed/prediction/2020_03_30.csv
/data/ebms/feed/prediction/2020_03_31.csv
/data/ebms/feed/prediction/2020_04_01.csv
/data/ebms/feed/prediction/2020_04

# Example data

In [11]:
!pwd

/home/budf/phuonglinh.le/Hermes/notebook


In [12]:
!tree ../

../
├── data
│   ├── raw
│   │   ├── 11
│   │   │   ├── paths.json
│   │   │   ├── prediction_09h.json
│   │   │   ├── routes.json
│   │   │   ├── stops.json
│   │   │   ├── timetables.json
│   │   │   ├── trips.json
│   │   │   └── vars.json
│   │   ├── 110320.zip
│   │   └── 2020_03_11.csv
│   └── stag
└── notebook
    └── preprocessing.ipynb

5 directories, 10 files


Lấy từ tập data mẫu

`!unzip ../data/raw/110320.zip -d ../data/raw`

In [13]:
!ls -alt ../data/raw/

total 886656
drwxr-xr-x 3 phuonglinh.le phuonglinh.le        72 Jan  9 22:53 .
-rw-r--r-- 1 phuonglinh.le phuonglinh.le 134773718 Jan  9 22:45 110320.zip
drwxr-xr-x 5 phuonglinh.le phuonglinh.le        71 Jan  9 22:38 ..
-rwxrwxrwx 1 phuonglinh.le phuonglinh.le 773157599 Oct 26 15:16 2020_03_11.csv
drwxrwxrwx 2 phuonglinh.le phuonglinh.le       178 Oct 18 22:18 11


In [60]:
!find ../* -type f -name "*.json" -o -name "*.csv"

../data/raw/11/paths.json
../data/raw/11/prediction_09h.json
../data/raw/11/routes.json
../data/raw/11/stops.json
../data/raw/11/timetables.json
../data/raw/11/trips.json
../data/raw/11/vars.json
../data/raw/2020_03_11.csv


### Example csv data

In [2]:
columns = ['requestTime','stopID','routeID',
           'routeVarID','busID','locTimeStamp','distance (m)',
           'speed (km/h)','timeToStop (sec)', '0']

Tập Dataset `B-Log` ghi nhận thông tin xe bus:
* Thời gian request dữ liệu: `requestTime`
* Thông tin kĩ thuật của xe bus: Đang ở route nào (`routeID`, `routeVarID`), `busID`,...
* Thời gian ghi nhận `locTimeStamp` của xe bus: **GMT+7**

In [3]:
data_202003 = pd.read_csv("../data/raw/2020_03_11.csv", dtype="str", names=columns)

In [4]:
data_202003.drop('0', inplace=True, axis=1)

In [5]:
data_202003.head()

,requestTime,stopID,routeID,routeVarID,busID,locTimeStamp,distance (m),speed (km/h),timeToStop (sec)
0,2020-03-11T04:00:02+07:00,7468,279,2,51B40062,2019-11-29T09:29:31+07:00,4960.11,20.0,891
1,2020-03-11T04:00:02+07:00,6861,178,1,51B40134,2020-03-10T21:58:34+07:00,7039.73,0.0,796
2,2020-03-11T04:00:02+07:00,6862,179,2,51B16060,2020-03-10T21:58:48+07:00,9176.89,0.0,1180
3,2020-03-11T04:00:02+07:00,6863,179,1,51B15961,2020-03-10T21:58:51+07:00,15299.00,0.0,1893
4,2020-03-11T04:00:02+07:00,725,81,2,51B28743,2020-03-10T19:46:42+07:00,228.72,34.0,55


In [6]:
data_202003.tail()

,requestTime,stopID,routeID,routeVarID,busID,locTimeStamp,distance (m),speed (km/h),timeToStop (sec)
8624289,2020-03-11T21:59:58+07:00,4086,137,274,51B06503,2020-03-11T21:59:13+07:00,11564.43,0.0,1722
8624290,2020-03-11T21:59:58+07:00,4086,137,274,51B06343,2020-03-11T21:59:22+07:00,11995.90,0.0,1789
8624291,2020-03-11T21:59:58+07:00,4086,137,274,51B06392,2020-03-11T21:59:37+07:00,12000.62,11.0,1789
8624292,2020-03-11T21:59:58+07:00,4086,137,274,51B06369,2020-03-11T21:59:19+07:00,12000.18,0.0,1792
8624293,2020-03-11T21:59:58+07:00,4086,137,274,51B06446,2020-03-11T21:59:23+07:00,12033.11,0.0,1794


In [7]:
data_202003.shape

(8624294, 9)

In [8]:
data_202003.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8624294 entries, 0 to 8624293
Data columns (total 9 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   requestTime       object
 1   stopID            object
 2   routeID           object
 3   routeVarID        object
 4   busID             object
 5   locTimeStamp      object
 6   distance (m)      object
 7   speed (km/h)      object
 8   timeToStop (sec)  object
dtypes: object(9)
memory usage: 592.2+ MB


### Read Json file

In [26]:
def read_dim_data(path, is_list_json=True):
    if is_list_json:
        vars_data = pd.read_json(path, lines=True).stack().reset_index(drop=True)

        vars_df = pd.json_normalize(vars_data[~vars_data.isna()])
    else:
        vars_df = pd.read_json(path, lines=True)
    
    return vars_df

def expand_column(stops_df, column):
    stops_df = stops_df.reset_index()

    stops_stag = pd.concat(
        [pd.DataFrame(d)
         for d 
         in stops_df[column]], 
        keys=stops_df["index"]
    ).reset_index(level=1, drop=True)

    stops_df = (
        stops_df.drop(["index"], axis=1)
            .join(stops_stag)
            .reset_index(drop=True)
    )
    
    return stops_df

In [10]:
!find ../* -type f -name "*.json" -o -name "*.csv"

../data/raw/11/paths.json
../data/raw/11/prediction_09h.json
../data/raw/11/routes.json
../data/raw/11/stops.json
../data/raw/11/timetables.json
../data/raw/11/trips.json
../data/raw/11/vars.json
../data/raw/2020_03_11.csv


Xem xét 1 vài dòng dữ liệu thô của tập **B-Info**:
* Là tập flat files với định dạng **like** json.
* Mỗi một dòng là một Json `array` / `object` được phân cách bằng dấu ngắt câu `\n`, có thể là dấu `,`.
* Tuỳ vào cách thể hiện mà xử lý đọc sẽ có thể khác nhau.

In [21]:
!tail -n 2 ../data/raw/11/timetables.json

[{"RouteId":318,"RouteVarId":1,"TimeTableId":1,"RouteVarShortName":"Trần Trọng Cung","StartDate":"05/12/2017","EndDate":"","StartStop":"TH Le Ngoc Han","EndStop":"Tran Trong Cung","IsCurrent":true,"RunningTime":"90","Headway":"0","TotalTrip":1,"OperationTime":"16:30 - 16:30","ApplyDates":"T2, T3, T4, T5, T6, T7"}]
[{"RouteId":313,"RouteVarId":1,"TimeTableId":1,"RouteVarShortName":"Trường Tiểu Học Nguyễn Bỉnh Khiêm","StartDate":"28/11/2017","EndDate":"","StartStop":"Thao Dien","EndStop":"TH Nguyen Bỉnh Khiem","IsCurrent":true,"RunningTime":"90","Headway":"0","TotalTrip":1,"OperationTime":"06:00 - 06:00","ApplyDates":"T2, T3, T4, T5, T6, T7, CN"},{"RouteId":313,"RouteVarId":2,"TimeTableId":2,"RouteVarShortName":"Thảo Điền","StartDate":"28/11/2017","EndDate":"","StartStop":"TH Nguyen Bỉnh Khiem","EndStop":"Thao Dien","IsCurrent":true,"RunningTime":"80","Headway":"0","TotalTrip":1,"OperationTime":"16:40 - 16:40","ApplyDates":"T2, T3, T4, T5, T6, T7, CN"}]


In [32]:
!tail -n 2 ../data/raw/11/routes.json

[{"RouteId":1,"RouteNo":"01","RouteName":"Bến Xe Buýt Sài Gòn - Bến Xe Buýt Chợ Lớn"},{"RouteId":2,"RouteNo":"02","RouteName":"Bến Xe Buýt Sài Gòn - Bến Xe Miền Tây"},{"RouteId":3,"RouteNo":"03","RouteName":"Bến Xe Buýt Sài Gòn - Bến Xe Buýt  Thạnh Lộc"},{"RouteId":4,"RouteNo":"04","RouteName":"Bến Xe Buýt Sài Gòn- Cộng Hòa - Bến Xe  An Sương"},{"RouteId":5,"RouteNo":"05","RouteName":"Bến Xe Buýt Chợ Lớn - Biên Hòa"},{"RouteId":6,"RouteNo":"06","RouteName":"Bến Xe Buýt Chợ Lớn- Đại học Nông Lâm"},{"RouteId":7,"RouteNo":"07","RouteName":"Bến Xe Buýt Chợ Lớn- Gò vấp"},{"RouteId":8,"RouteNo":"08","RouteName":"Bến Xe Buýt Quận 8- Đại học Quốc Gia"},{"RouteId":9,"RouteNo":"09","RouteName":"Bến Xe Buýt Chợ Lớn - Bình Chánh - Hưng Long"},{"RouteId":10,"RouteNo":"10","RouteName":"Đại học Quốc Gia- Bến xe Miền Tây"},{"RouteId":11,"RouteNo":"100","RouteName":"Bến Xe Buýt Củ Chi - Cầu Tân Thái"},{"RouteId":12,"RouteNo":"101","RouteName":"Bến Xe Buýt Chợ Lớn - Chợ Tân Nhựt"},{"RouteId":13,"RouteNo

In [34]:
!tail -n 2 ../data/raw/11/vars.json

[{"RouteId":318,"RouteVarId":1,"RouteVarName":"Lượt về","RouteVarShortName":"Trần Trọng Cung","RouteNo":"HS-98","StartStop":"TH Le Ngoc Han","EndStop":"Tran Trong Cung","Distance":12541.0,"Outbound":true,"RunningTime":90}]
[{"RouteId":316,"RouteVarId":1,"RouteVarName":"Lượt đi","RouteVarShortName":"Trường Tiểu Học Nguyễn Bỉnh Khiêm","RouteNo":"HS-96","StartStop":"Phan Van Tri","EndStop":"TH Nguyen Bỉnh Khiem","Distance":17414.0,"Outbound":false,"RunningTime":90},{"RouteId":316,"RouteVarId":2,"RouteVarName":"Lượt về","RouteVarShortName":"Phan Văn Trị","RouteNo":"HS-96","StartStop":"TH Nguyen Bỉnh Khiem","EndStop":"Phan Van Tri","Distance":9080.0,"Outbound":true,"RunningTime":80}]


In [35]:
!tail -n 2 ../data/raw/11/trips.json

[{"RouteId":313,"TripId":1,"TimeTableId":1,"StartTime":"06:00","EndTime":"07:30"}]
[{"RouteId":313,"TripId":2,"TimeTableId":2,"StartTime":"16:40","EndTime":"18:00"}]


In [37]:
!tail -n 3 ../data/raw/11/stops.json

{"Stops":[{"StopId":7512,"Code":"BX_THnguyenbinhkhiem","Name":"TH Nguyen Bỉnh Khiem","StopType":"Bến xe","Zone":"Học Sinh","Ward":null,"AddressNo":"TH nguyen binh khiem","Street":"TH nguyen binh khiem","SupportDisability":"Có","Status":"Đang khai thác","Lng":106.704143,"Lat":10.788183,"Search":"TNBK Tnbk Tnbk","Routes":"HS-92, HS-93, HS-94, HS-95, HS-96"},{"StopId":7519,"Code":"BX_phanvantri","Name":"Phan Van Tri","StopType":"Bến xe","Zone":"Học Sinh","Ward":null,"AddressNo":"phan van tri","Street":"phan van tri","SupportDisability":"Có","Status":"Đang khai thác","Lng":106.696279,"Lat":10.821727,"Search":"PVT pvt pvt","Routes":"HS-96"}],"RouteId":"316","RouteVarId":"2"}
{"Stops":[{"StopId":7513,"Code":"BX_THlengochan","Name":"TH Le Ngoc Han","StopType":"Bến xe","Zone":"Học Sinh","Ward":null,"AddressNo":"TH le ngoc han","Street":"TH le ngoc han","SupportDisability":"Có","Status":"Đang khai thác","Lng":106.689983,"Lat":10.772544,"Search":"TLNH Tlnh Tlnh","Routes":"HS-98"},{"StopId":7520,

In [39]:
!tail -n 1 ../data/raw/11/paths.json

{"lat":[10.80461407,10.80642319,10.80811501,10.80897903,10.80941105,10.80991173,10.810359,10.81191349,10.81417465,10.81451702,10.81327343,10.8139267,10.81400013,10.81337833,10.81459045,10.81514931,10.81643486,10.81708813,10.81734085,10.81740475,10.81734657,10.81714058,10.81665611,10.81479645,10.81081295,10.81048107,10.80923176,10.80783558,10.80703449,10.80570698,10.80503178,10.80434227,10.80386734,10.8025341,10.80227089,10.80196571,10.80109119,10.7980032,10.79809284,10.79831886,10.7986145,10.7988615,10.79903603,10.79897213,10.7988987,10.79866695,10.79791832,10.7976656,10.79760742,10.7976656,10.79761314,10.79654884,10.79611111,10.7969389,10.79576874,10.79543114,10.79646397,10.79604244,10.79528427,10.79300785,10.79129982,10.79073048,10.79017258,10.78939247,10.78912354,10.78973007,10.7893343,10.78824329,10.7863884,10.78532982,10.78465462,10.78652096,10.78812218,10.78818321],"lng":[106.72650146,106.72585297,106.72517395,106.72483063,106.72473145,106.7247467,106.72490692,106.72550201,106.72

In [27]:
# Đọc tập dữ liệu 
timetables_df = read_dim_data("../data/raw/11/timetables.json")
routes_df = read_dim_data("../data/raw/11/routes.json")
vars_df = read_dim_data("../data/raw/11/vars.json")
trips_df = read_dim_data("../data/raw/11/trips.json")

stops_df = expand_column(read_dim_data("../data/raw/11/stops.json", False), "Stops")

paths_df = (
    read_dim_data("../data/raw/11/paths.json", False)
        .set_index(["RouteId", "RouteVarId"])
        .apply(pd.Series.explode)
        .reset_index()
)
paths_df.loc[:, ["lat", "lng"]] = paths_df[["lat", "lng"]].astype(float)

In [40]:
routes_df.head()

,RouteId,RouteNo,RouteName
0,1,01,Bến Xe Buýt Sài Gòn - Bến Xe Buýt Chợ Lớn
1,2,02,Bến Xe Buýt Sài Gòn - Bến Xe Miền Tây
2,3,03,Bến Xe Buýt Sài Gòn - Bến Xe Buýt Thạnh Lộc
3,4,04,Bến Xe Buýt Sài Gòn- Cộng Hòa - Bến Xe An Sương
4,5,05,Bến Xe Buýt Chợ Lớn - Biên Hòa


In [62]:
vars_df.head()

,RouteId,RouteVarId,RouteVarName,RouteVarShortName,RouteNo,StartStop,EndStop,Distance,Outbound,RunningTime
0,1,1,Lượt đi: Bến Thành - BX Chợ Lớn,Bến Xe Buýt Chợ Lớn,01,Công Trường Mê Linh,Bến xe Chợ Lớn,8424.0,True,40
1,1,2,Lượt về: BX Chợ Lớn - Bến Thành,Bến Xe Buýt Sài Gòn,01,Bến xe Chợ Lớn,Công Trường Mê Linh,9612.0,False,40
2,8,15,Lượt đi: Bến xe Quận 8 - Đại học Quốc Gia,Đại học Quốc Gia,08,Bến xe buýt Quận 8,Đại học Quốc gia,31772.0,True,90
3,8,16,Lượt về: Đại học Quốc Gia - Bến xe Quận 8,Bến Xe Buýt Quận 8,08,Đại học Quốc gia,Bến xe buýt Quận 8,31940.0,False,90
4,2,1,Công Trường Mê Linh - Bến xe Miền Tây,Bến Xe Miền Tây,02,Công Trường Mê Linh,Bến xe Miền Tây,13148.0,True,50


In [378]:
paths_df[(paths_df.RouteId == 81) & (paths_df.RouteVarId == 2)]

,RouteId,RouteVarId,lat,lng
42654,81,2,10.954127,106.808952
42655,81,2,10.954416,106.808601
42656,81,2,10.955017,106.809235
42657,81,2,10.955502,106.809921
42658,81,2,10.955934,106.811050
...,...,...,...,...
43140,81,2,10.740655,106.618263
43141,81,2,10.740705,106.618317
43142,81,2,10.740705,106.618317
43143,81,2,10.740705,106.618317


In [41]:
timetables_df.head()

,RouteId,RouteVarId,TimeTableId,RouteVarShortName,StartDate,EndDate,StartStop,EndStop,IsCurrent,RunningTime,Headway,TotalTrip,OperationTime,ApplyDates
0,1,1,31,Bến Xe Buýt Chợ Lớn,15/03/2020,,Công Trường Mê Linh,Bến xe Chợ Lớn,False,35,10,103,05:00 - 20:30,"T2, T3, T4, T5, T6, T7, CN"
1,1,2,32,Bến Xe Buýt Sài Gòn,15/03/2020,,Bến xe Chợ Lớn,Công Trường Mê Linh,False,35,10,103,05:00 - 20:30,"T2, T3, T4, T5, T6, T7, CN"
2,1,1,33,Bến Xe Buýt Chợ Lớn,10/02/2020,,Công Trường Mê Linh,Bến xe Chợ Lớn,True,35,12 - 15,75,05:00 - 20:30,"T2, T3, T4, T5, T6, T7, CN"
3,1,2,34,Bến Xe Buýt Sài Gòn,10/02/2020,,Bến xe Chợ Lớn,Công Trường Mê Linh,True,35,12 - 15,75,05:00 - 20:30,"T2, T3, T4, T5, T6, T7, CN"
4,9,1,27,Hưng Long,10/02/2020,,Bến xe Chợ Lớn,Hưng Long,True,70,8 - 10,100,05:25 - 19:10,"T2, T3, T4, T5, T6, T7, CN"


Lấy thông tin từ các routes bổ sung vào tập `vars`

In [43]:
vars_pre = vars_df.merge(
    routes_df[["RouteId", "RouteName"]],
    on="RouteId",
    how="left",
    suffixes=("", "_y")
)

In [44]:
vars_pre.head()

,RouteId,RouteVarId,RouteVarName,RouteVarShortName,RouteNo,StartStop,EndStop,Distance,Outbound,RunningTime,RouteName
0,1,1,Lượt đi: Bến Thành - BX Chợ Lớn,Bến Xe Buýt Chợ Lớn,01,Công Trường Mê Linh,Bến xe Chợ Lớn,8424.0,True,40,Bến Xe Buýt Sài Gòn - Bến Xe Buýt Chợ Lớn
1,1,2,Lượt về: BX Chợ Lớn - Bến Thành,Bến Xe Buýt Sài Gòn,01,Bến xe Chợ Lớn,Công Trường Mê Linh,9612.0,False,40,Bến Xe Buýt Sài Gòn - Bến Xe Buýt Chợ Lớn
2,8,15,Lượt đi: Bến xe Quận 8 - Đại học Quốc Gia,Đại học Quốc Gia,08,Bến xe buýt Quận 8,Đại học Quốc gia,31772.0,True,90,Bến Xe Buýt Quận 8- Đại học Quốc Gia
3,8,16,Lượt về: Đại học Quốc Gia - Bến xe Quận 8,Bến Xe Buýt Quận 8,08,Đại học Quốc gia,Bến xe buýt Quận 8,31940.0,False,90,Bến Xe Buýt Quận 8- Đại học Quốc Gia
4,2,1,Công Trường Mê Linh - Bến xe Miền Tây,Bến Xe Miền Tây,02,Công Trường Mê Linh,Bến xe Miền Tây,13148.0,True,50,Bến Xe Buýt Sài Gòn - Bến Xe Miền Tây


In [45]:
# Timetables dự kiến
timetables_pre = timetables_df.merge(
    vars_pre[["RouteId", "RouteVarId", "Distance", "Outbound"]],
    on=["RouteId", "RouteVarId"],
    how="left",
)

In [46]:
timetables_pre.sort_values(["RouteId", "RouteVarId", "TimeTableId"]).head(10)

,RouteId,RouteVarId,TimeTableId,RouteVarShortName,StartDate,EndDate,StartStop,EndStop,IsCurrent,RunningTime,Headway,TotalTrip,OperationTime,ApplyDates,Distance,Outbound
0,1,1,31,Bến Xe Buýt Chợ Lớn,15/03/2020,,Công Trường Mê Linh,Bến xe Chợ Lớn,False,35,10,103,05:00 - 20:30,"T2, T3, T4, T5, T6, T7, CN",8424.0,True
2,1,1,33,Bến Xe Buýt Chợ Lớn,10/02/2020,,Công Trường Mê Linh,Bến xe Chợ Lớn,True,35,12 - 15,75,05:00 - 20:30,"T2, T3, T4, T5, T6, T7, CN",8424.0,True
1,1,2,32,Bến Xe Buýt Sài Gòn,15/03/2020,,Bến xe Chợ Lớn,Công Trường Mê Linh,False,35,10,103,05:00 - 20:30,"T2, T3, T4, T5, T6, T7, CN",9612.0,False
3,1,2,34,Bến Xe Buýt Sài Gòn,10/02/2020,,Bến xe Chợ Lớn,Công Trường Mê Linh,True,35,12 - 15,75,05:00 - 20:30,"T2, T3, T4, T5, T6, T7, CN",9612.0,False
8,2,1,7,Bến Xe Miền Tây,10/02/2020,,Công Trường Mê Linh,Bến xe Miền Tây,True,50,12,50,04:45 - 18:30,"T2, T3, T4, T5, T6, T7, CN",13148.0,True
10,2,1,11,Bến Xe Miền Tây,15/03/2020,,Công Trường Mê Linh,Bến xe Miền Tây,False,45 - 50,10 - 15,78,04:45 - 18:30,"T2, T3, T4, T5, T6, T7, CN",13148.0,True
9,2,2,8,Bến Xe Buýt Sài Gòn,10/02/2020,,Bến xe Miền Tây,Công Trường Mê Linh,True,50,12,50,04:45 - 18:30,"T2, T3, T4, T5, T6, T7, CN",12740.0,False
11,2,2,12,Bến Xe Buýt Sài Gòn,15/03/2020,,Bến xe Miền Tây,Công Trường Mê Linh,False,45 - 50,10 - 15,78,04:45 - 18:30,"T2, T3, T4, T5, T6, T7, CN",12740.0,False
36,3,5,33,Bến Xe Buýt Thạnh Lộc,30/01/2020,,Bến xe buýt Sài Gòn,THẠNH LỘC,True,60,8,108,04:55 - 20:45,"T2, T3, T4, T5, T6, T7, CN",17481.0,True
38,3,5,43,Bến Xe Buýt Thạnh Lộc,15/03/2020,,Bến xe buýt Sài Gòn,THẠNH LỘC,False,50 - 65,6 - 10,145,04:55 - 20:45,"T2, T3, T4, T5, T6, T7, CN",17481.0,True


In [48]:
trips_df.head()

,RouteId,TripId,TimeTableId,StartTime,EndTime
0,1,4582,34,05:00,05:35
1,1,4583,34,05:15,05:50
2,1,4584,34,05:30,06:05
3,1,4585,34,05:45,06:20
4,1,4586,34,06:00,06:35


Thông tin của các stops:
* Nằm ở Route nào.
* Lng Lat.
* Distance: tổng độ dài của RouteVarld.

In [51]:
stop_pre = stops_df.merge(
    vars_pre,
    on=["RouteId", "RouteVarId"],
    how="left",
).drop("Stops", axis=1).drop_duplicates()

In [52]:
stop_pre.head()

,RouteId,RouteVarId,StopId,Code,Name,StopType,Zone,Ward,AddressNo,Street,SupportDisability,Status,Lng,Lat,Search,Routes,RouteVarName,RouteVarShortName,RouteNo,StartStop,EndStop,Distance,Outbound,RunningTime,RouteName
0,1,1,33,BX 06,Công Trường Mê Linh,Bến xe,Quận 1,None,Công trường Mê Linh,Thi Sách,,Đang khai thác,106.705856,10.776790,CTML CtML TS,"01, 02",Lượt đi: Bến Thành - BX Chợ Lớn,Bến Xe Buýt Chợ Lớn,01,Công Trường Mê Linh,Bến xe Chợ Lớn,8424.0,True,40,Bến Xe Buýt Sài Gòn - Bến Xe Buýt Chợ Lớn
1,1,1,79,Q1 031,Bến Bạch Đằng,Trụ dừng,Quận 1,None,21,Tôn Đức Thắng,,Đang khai thác,106.706359,10.773299,BBD 21 TDT,"01, 02, 03, 120, 19, 45, 53, 56, 61-6, 88, D1,...",Lượt đi: Bến Thành - BX Chợ Lớn,Bến Xe Buýt Chợ Lớn,01,Công Trường Mê Linh,Bến xe Chợ Lớn,8424.0,True,40,Bến Xe Buýt Sài Gòn - Bến Xe Buýt Chợ Lớn
2,1,1,76,Q1 020,Cục Hải Quan Thành Phố,Nhà chờ,Quận 1,None,2-4,Hàm Nghi,,Đang khai thác,106.705549,10.770885,CHQTP 2-4 HN,"01, 02, 03, 123, 124, 19, 45, 56, 61-6, 88, D1",Lượt đi: Bến Thành - BX Chợ Lớn,Bến Xe Buýt Chợ Lớn,01,Công Trường Mê Linh,Bến xe Chợ Lớn,8424.0,True,40,Bến Xe Buýt Sài Gòn - Bến Xe Buýt Chợ Lớn
3,1,1,3691,Q1 193,Chợ Cũ,Nhà chờ,Quận 1,None,84,Hàm Nghi,,Đang khai thác,106.703108,10.771006,CC 84 HN,"01, 02, 03, 19, 45, 56, 86, 88",Lượt đi: Bến Thành - BX Chợ Lớn,Bến Xe Buýt Chợ Lớn,01,Công Trường Mê Linh,Bến xe Chợ Lớn,8424.0,True,40,Bến Xe Buýt Sài Gòn - Bến Xe Buýt Chợ Lớn
4,1,1,2028,Q1 194,Trường Cao Thắng,Nhà chờ,Quận 1,None,Trường Cao Thắng,Hàm Nghi,,Đang khai thác,106.701853,10.771043,TCT TCT HN,"01, 02, 03, 102, 11, 19, 20, 34, 38, 39, 45, 5...",Lượt đi: Bến Thành - BX Chợ Lớn,Bến Xe Buýt Chợ Lớn,01,Công Trường Mê Linh,Bến xe Chợ Lớn,8424.0,True,40,Bến Xe Buýt Sài Gòn - Bến Xe Buýt Chợ Lớn


In [ ]:
stop_pre = stop_pre.merge(
    stop_pre.groupby(["RouteId", "RouteVarId"]).agg(NumStop=("StopId", "nunique")),
    on=["RouteId", "RouteVarId"],
)

In [77]:
stop_pre.head()

,RouteId,RouteVarId,StopId,Code,Name,StopType,Zone,Ward,AddressNo,Street,SupportDisability,Status,Lng,Lat,Search,Routes,RouteVarName,RouteVarShortName,RouteNo,StartStop,EndStop,Distance,Outbound,RunningTime,RouteName,NumStop
0,1,1,33,BX 06,Công Trường Mê Linh,Bến xe,Quận 1,None,Công trường Mê Linh,Thi Sách,,Đang khai thác,106.705856,10.776790,CTML CtML TS,"01, 02",Lượt đi: Bến Thành - BX Chợ Lớn,Bến Xe Buýt Chợ Lớn,01,Công Trường Mê Linh,Bến xe Chợ Lớn,8424.0,True,40,Bến Xe Buýt Sài Gòn - Bến Xe Buýt Chợ Lớn,29
1,1,1,79,Q1 031,Bến Bạch Đằng,Trụ dừng,Quận 1,None,21,Tôn Đức Thắng,,Đang khai thác,106.706359,10.773299,BBD 21 TDT,"01, 02, 03, 120, 19, 45, 53, 56, 61-6, 88, D1,...",Lượt đi: Bến Thành - BX Chợ Lớn,Bến Xe Buýt Chợ Lớn,01,Công Trường Mê Linh,Bến xe Chợ Lớn,8424.0,True,40,Bến Xe Buýt Sài Gòn - Bến Xe Buýt Chợ Lớn,29
2,1,1,76,Q1 020,Cục Hải Quan Thành Phố,Nhà chờ,Quận 1,None,2-4,Hàm Nghi,,Đang khai thác,106.705549,10.770885,CHQTP 2-4 HN,"01, 02, 03, 123, 124, 19, 45, 56, 61-6, 88, D1",Lượt đi: Bến Thành - BX Chợ Lớn,Bến Xe Buýt Chợ Lớn,01,Công Trường Mê Linh,Bến xe Chợ Lớn,8424.0,True,40,Bến Xe Buýt Sài Gòn - Bến Xe Buýt Chợ Lớn,29
3,1,1,3691,Q1 193,Chợ Cũ,Nhà chờ,Quận 1,None,84,Hàm Nghi,,Đang khai thác,106.703108,10.771006,CC 84 HN,"01, 02, 03, 19, 45, 56, 86, 88",Lượt đi: Bến Thành - BX Chợ Lớn,Bến Xe Buýt Chợ Lớn,01,Công Trường Mê Linh,Bến xe Chợ Lớn,8424.0,True,40,Bến Xe Buýt Sài Gòn - Bến Xe Buýt Chợ Lớn,29
4,1,1,2028,Q1 194,Trường Cao Thắng,Nhà chờ,Quận 1,None,Trường Cao Thắng,Hàm Nghi,,Đang khai thác,106.701853,10.771043,TCT TCT HN,"01, 02, 03, 102, 11, 19, 20, 34, 38, 39, 45, 5...",Lượt đi: Bến Thành - BX Chợ Lớn,Bến Xe Buýt Chợ Lớn,01,Công Trường Mê Linh,Bến xe Chợ Lớn,8424.0,True,40,Bến Xe Buýt Sài Gòn - Bến Xe Buýt Chợ Lớn,29


In [73]:
stop_pre.groupby(["RouteId", "RouteVarId"]).agg(NumStop=("StopId", "nunique"))

NumStop
RouteId RouteVarId         
1       1                29
        2                33
2       1                42
        2                40
3       5                47
        6                51
4       7                47
        8                39
5       1                69
        2                59
6       1                51
        2                61
7       13               44
        14               50
8       15               68
        16               65
9       1                60
        2                59
10      19               15
        20               14
11      1                27
        2                27
12      1                35
        2                36
13      25               60
        26               62
14      1                58
        2                60
15      29               63
        30               61
16      31               23
        32               23
17      1                34
        2                26
18      1                43
        2                46
20      39               36
        40               37
21      41               15
        42               16
22      43               22
        44               19
23      45               37
        46               34
24      47               26
        48               27
25      1                57
        2                62
26      1                48
        2                42
27      53               46
        54               43
28      55               46
        56               46
29      1                42
        2                46
30      59               58
        60               62
31      63               42
        64               45
32      1                41
        2                52
34      67               36
        68               37
35      69               51
        70               53
36      1                37
        2                33
37      73               39
        74               34
38      77               41
        78               42
39      1                40
        2                41
40      79               65
        80               64
41      81               53
        82               59
42      83               50
        84               47
43      85               47
        86               47
44      87               57
        88               58
45      89               68
        90               63
46      91               49
        92               47
48      95               42
        96               39
49      97               66
        98               64
50      99               43
        100              51
51      2                64
        3                62
52      1                60
        2                55
53      105              70
        106              69
54      1                47
        2                53
55      1                48
        2                50
57      113              51
        114              52
59      117              42
        118              44
60      3                34
        4                33
62      123              54
        124              53
64      127              32
        128              31
65      129              40
        130              44
66      131              47
        132              48
67      133              34
        134              47
68      1                37
        2                38
69      1                56
        2                56
70      1                49
        2                48
72      143              39
        144              36
73      1                17
        2                18
75      149              70
        150              62
76      151              60
        152              59
77      153              29
        154              29
78      1                37
        2                37
79      157              54
        158              64
81      1                78
        2       

# Full Data - Old Data

In [53]:
_convert_col = ["stopID", "routeID", "routeVarID"]
data_202003[_convert_col] = data_202003[_convert_col].astype(int)

In [54]:
data_202003_pre = data_202003.merge(
    stop_pre,
    right_on=["RouteId", "RouteVarId", "StopId"],
    left_on=["routeID", "routeVarID", "stopID"],
    how="left",
)

In [55]:
data_202003_pre.requestTime = pd.to_datetime(data_202003_pre.requestTime)
data_202003_pre.locTimeStamp = pd.to_datetime(data_202003_pre.locTimeStamp)

In [56]:
data_202003_pre.columns

Index(['requestTime', 'stopID', 'routeID', 'routeVarID', 'busID',
       'locTimeStamp', 'distance (m)', 'speed (km/h)', 'timeToStop (sec)',
       'RouteId', 'RouteVarId', 'StopId', 'Code', 'Name', 'StopType', 'Zone',
       'Ward', 'AddressNo', 'Street', 'SupportDisability', 'Status', 'Lng',
       'Lat', 'Search', 'Routes', 'RouteVarName', 'RouteVarShortName',
       'RouteNo', 'StartStop', 'EndStop', 'Distance', 'Outbound',
       'RunningTime', 'RouteName'],
      dtype='object')

In [57]:
_col = ["distance (m)","speed (km/h)", "timeToStop (sec)"]
data_202003_pre[_col] = data_202003_pre[_col].astype(float)

In [58]:
data_202003_sorted = data_202003_pre.sort_values(["busID", "locTimeStamp"])

In [59]:
data_202003_sorted[(data_202003_sorted == 7178).any(axis=1)]

,requestTime,stopID,routeID,routeVarID,busID,locTimeStamp,distance (m),speed (km/h),timeToStop (sec),RouteId,RouteVarId,StopId,Code,Name,StopType,Zone,Ward,AddressNo,Street,SupportDisability,Status,Lng,Lat,Search,Routes,RouteVarName,RouteVarShortName,RouteNo,StartStop,EndStop,Distance,Outbound,RunningTime,RouteName
3201047,2020-03-11 10:54:54+07:00,7280,215,1,51B04302,2020-03-11 10:54:37+07:00,68375.88,49.0,7178.0,215,1,7280,BXLT Phu Tuc,Bến xe Phú Túc,Bến xe,Ngoại thành,None,Bến xe Phú Túc,Quốc lộ 20,Có,Đang khai thác,107.217992,11.092120,BxPT BxPT Ql20,60-2,lượt đi,Bến Xe Phú Túc,60-2,Đại học Nông Lâm,Bến xe Phú Túc,70454.0,True,120,Đại Học Nông Lâm - Bến Xe Phú Túc
422,2020-03-11 04:00:12+07:00,7178,210,1,51HC00008,2020-03-10 21:58:56+07:00,13.01,1.0,5.0,210,1,7178,Q1 D1,Thảo cầm viên,Bến xe,Quận 1,Phường Bến Nghé,2,Nguyễn Bỉnh Khiêm,Có,Đang khai thác,106.705243,10.787311,Tcv 2 NBK,D1,Lượt đi,THẢO CẦM VIÊN,D1,Bến xe buýt Sài Gòn,Thảo cầm viên,6264.0,True,30,CÔNG VIÊN 23/9-THẢO CẦM VIÊN
1176,2020-03-11 04:00:22+07:00,7178,210,1,51HC00008,2020-03-10 21:58:56+07:00,13.01,1.0,5.0,210,1,7178,Q1 D1,Thảo cầm viên,Bến xe,Quận 1,Phường Bến Nghé,2,Nguyễn Bỉnh Khiêm,Có,Đang khai thác,106.705243,10.787311,Tcv 2 NBK,D1,Lượt đi,THẢO CẦM VIÊN,D1,Bến xe buýt Sài Gòn,Thảo cầm viên,6264.0,True,30,CÔNG VIÊN 23/9-THẢO CẦM VIÊN
1826,2020-03-11 04:00:32+07:00,7178,210,1,51HC00008,2020-03-10 21:58:56+07:00,13.01,1.0,5.0,210,1,7178,Q1 D1,Thảo cầm viên,Bến xe,Quận 1,Phường Bến Nghé,2,Nguyễn Bỉnh Khiêm,Có,Đang khai thác,106.705243,10.787311,Tcv 2 NBK,D1,Lượt đi,THẢO CẦM VIÊN,D1,Bến xe buýt Sài Gòn,Thảo cầm viên,6264.0,True,30,CÔNG VIÊN 23/9-THẢO CẦM VIÊN
2641,2020-03-11 04:00:42+07:00,7178,210,1,51HC00008,2020-03-10 21:58:56+07:00,13.01,1.0,5.0,210,1,7178,Q1 D1,Thảo cầm viên,Bến xe,Quận 1,Phường Bến Nghé,2,Nguyễn Bỉnh Khiêm,Có,Đang khai thác,106.705243,10.787311,Tcv 2 NBK,D1,Lượt đi,THẢO CẦM VIÊN,D1,Bến xe buýt Sài Gòn,Thảo cầm viên,6264.0,True,30,CÔNG VIÊN 23/9-THẢO CẦM VIÊN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3075453,2020-03-11 10:41:13+07:00,8,92,2,62B01282,2020-03-11 10:40:41+07:00,62825.69,33.0,7178.0,92,2,8,BX14,Bến xe Chợ Lớn,Bến xe,Quận 5,None,GA HKXB CHO LON,Lê Quang Sung,,Đang khai thác,106.652565,10.751253,BxCL GHCL LQS,"01, 05, 06, 07, 09, 150, 23, 47, 56, 62-1, 62-...",Lượt về,Bến xe Chợ Lớn,62-10,Bến xe Thanh Vĩnh Đông,Bến xe Chợ Lớn,67667.0,False,140,Bến xe Chợ Lớn - Thanh Vĩnh Đông
3077042,2020-03-11 10:41:23+07:00,8,92,2,62B01282,2020-03-11 10:40:41+07:00,62825.69,33.0,7178.0,92,2,8,BX14,Bến xe Chợ Lớn,Bến xe,Quận 5,None,GA HKXB CHO LON,Lê Quang Sung,,Đang khai thác,106.652565,10.751253,BxCL GHCL LQS,"01, 05, 06, 07, 09, 150, 23, 47, 56, 62-1, 62-...",Lượt về,Bến xe Chợ Lớn,62-10,Bến xe Thanh Vĩnh Đông,Bến xe Chợ Lớn,67667.0,False,140,Bến xe Chợ Lớn - Thanh Vĩnh Đông
3078523,2020-03-11 10:41:33+07:00,8,92,2,62B01282,2020-03-11 10:40:41+07:00,62825.69,33.0,7178.0,92,2,8,BX14,Bến xe Chợ Lớn,Bến xe,Quận 5,None,GA HKXB CHO LON,Lê Quang Sung,,Đang khai thác,106.652565,10.751253,BxCL GHCL LQS,"01, 05, 06, 07, 09, 150, 23, 47, 56, 62-1, 62-...",Lượt về,Bến xe Chợ Lớn,62-10,Bến xe Thanh Vĩnh Đông,Bến xe Chợ Lớn,67667.0,False,140,Bến xe Chợ Lớn - Thanh Vĩnh Đông
3080054,2020-03-11 10:41:43+07:00,8,92,2,62B01282,2020-03-11 10:40:41+07:00,62825.69,33.0,7178.0,92,2,8,BX14,Bến xe Chợ Lớn,Bến xe,Quận 5,None,GA HKXB CHO LON,Lê Quang Sung,,Đang khai thác,106.652565,10.751253,BxCL GHCL LQS,"01, 05, 06, 07, 09, 150, 23, 47, 56, 62-1, 62-...",Lượt về,Bến xe Chợ Lớn,62-10,Bến xe Thanh Vĩnh Đông,Bến xe Chợ Lớn,67667.0,False,140,Bến xe Chợ Lớn - Thanh Vĩnh Đông


Here, StopID là stop cuối cùng của tuyến

# Prediction

In [436]:
COLUMN_HEADERS = ['RequestTime', 'StopId', 'RouteId', 'RouteVarId', 'BusId',
                  'LocTimeStamp', 'Distance', 'Speed', 'TimeToStop',
                  'Unnamed']

In [437]:
predic_org = pd.read_csv("/data/ebms/feed/prediction/2021_01_20.csv", 
                         dtype="str", names=COLUMN_HEADERS)

In [438]:
predic_df = predic_org.copy()

In [439]:
predic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5184941 entries, 0 to 5184940
Data columns (total 10 columns):
RequestTime     object
StopId          object
RouteId         object
RouteVarId      object
BusId           object
LocTimeStamp    object
Distance        object
Speed           object
TimeToStop      object
Unnamed         object
dtypes: object(10)
memory usage: 395.6+ MB


In [443]:
int_columns = ["StopId", "RouteVarId", "RouteId"]

float_columns = ["Distance", "Speed", "TimeToStop"]

In [444]:
predic_df

,RequestTime,StopId,RouteId,RouteVarId,BusId,LocTimeStamp,Distance,Speed,TimeToStop,Unnamed
0,2021-01-20T03:00:31+07:00,1389,114,228,51B23483,2021-01-20T03:00:31+07:00,7524.00,51,713,0
1,2021-01-20T03:00:31+07:00,166,114,228,51B23483,2021-01-20T03:00:31+07:00,17121.00,51,1839,0
2,2021-01-20T03:00:41+07:00,1389,114,228,51B23483,2021-01-20T03:00:41+07:00,7524.00,51,703,0
3,2021-01-20T03:00:41+07:00,166,114,228,51B23483,2021-01-20T03:00:41+07:00,17121.00,51,1829,0
4,2021-01-20T03:00:51+07:00,1389,114,228,51B23483,2021-01-20T03:00:51+07:00,7524.00,51,693,0
...,...,...,...,...,...,...,...,...,...,...
5184936,2021-01-20T21:28:10+07:00,2272,45,89,51B30409,2021-01-20T21:28:10+07:00,163.00,34,116,0
5184937,2021-01-20T21:28:20+07:00,2272,45,89,51B30409,2021-01-20T21:28:20+07:00,163.00,34,106,0
5184938,2021-01-20T21:28:30+07:00,2272,45,89,51B30409,2021-01-20T21:28:30+07:00,163.00,34,96,0
5184939,2021-01-20T21:28:40+07:00,2272,45,89,51B30409,2021-01-20T21:28:40+07:00,163.00,34,86,0


In [481]:
predic_df.BusId.unique()[:20]

array(['51B23483', '51B30700', '51B08991', '53N5867', '51B24250',
       '51B01342', '51B16642', '53N4527', '53N4498', '51B30764',
       '51B16534', '53N3437', '53N5214', '51B23990', '51B23888',
       '51B16531', '51B24127', '51B23865', '53N3351', '53N5206'],
      dtype=object)

In [445]:
predic_df[int_columns] = predic_df.loc[:, int_columns].astype("float").astype("int")

predic_df[float_columns] = predic_df.loc[:, float_columns].astype("float")

In [447]:
example = predic_df[predic_df.BusId == "51B23483"].copy()

In [475]:
predic_df[predic_df.BusId == "51B30409"].drop_duplicates(["RouteId", "RouteVarId"])

,RequestTime,StopId,RouteId,RouteVarId,BusId,LocTimeStamp,Distance,Speed,TimeToStop,Unnamed
64834,2021-01-20T04:57:34+07:00,2342,45,90,51B30409,2021-01-20T04:57:34+07:00,6799.0,23.0,1052.0,0
511306,2021-01-20T06:21:48+07:00,590,45,89,51B30409,2021-01-20T06:21:48+07:00,2459.0,0.0,466.0,0


In [478]:
predic_df[predic_df.BusId == "53N4498"].drop_duplicates(["RouteId", "RouteVarId"])

,RequestTime,StopId,RouteId,RouteVarId,BusId,LocTimeStamp,Distance,Speed,TimeToStop,Unnamed
632,2021-01-20T03:32:32+07:00,35,24,48,53N4498,2021-01-20T03:32:32+07:00,34857.0,3.0,4368.0,0
325643,2021-01-20T05:51:26+07:00,732,24,47,53N4498,2021-01-20T05:51:26+07:00,34128.0,0.0,3957.0,0


In [482]:
predic_df[predic_df.BusId == "53N5206"].drop_duplicates(["RouteId", "RouteVarId"])

,RequestTime,StopId,RouteId,RouteVarId,BusId,LocTimeStamp,Distance,Speed,TimeToStop,Unnamed
1875,2021-01-20T03:50:03+07:00,8,134,268,53N5206,2021-01-20T03:50:03+07:00,31510.0,54.0,4493.0,0
144523,2021-01-20T05:18:35+07:00,546,134,267,53N5206,2021-01-20T05:18:35+07:00,2754.0,6.0,1018.0,0


In [448]:
stop_pre.columns

Index(['RouteId', 'RouteVarId', 'StopId', 'Code', 'Name', 'StopType', 'Zone',
       'Ward', 'AddressNo', 'Street', 'SupportDisability', 'Status', 'Lng',
       'Lat', 'Search', 'Routes', 'RouteVarName', 'RouteVarShortName',
       'RouteNo', 'StartStop', 'EndStop', 'Distance', 'Outbound',
       'RunningTime', 'RouteName'],
      dtype='object')

In [449]:
example.drop_duplicates(["RouteId", "RouteVarId"])

,RequestTime,StopId,RouteId,RouteVarId,BusId,LocTimeStamp,Distance,Speed,TimeToStop,Unnamed
0,2021-01-20T03:00:31+07:00,1389,114,228,51B23483,2021-01-20T03:00:31+07:00,7524.0,51.0,713.0,0
4890,2021-01-20T04:05:41+07:00,946,53,105,51B23483,2021-01-20T04:05:41+07:00,24256.0,29.0,4520.0,0
541770,2021-01-20T06:26:38+07:00,4737,53,106,51B23483,2021-01-20T06:26:38+07:00,3220.0,0.0,565.0,0
5174033,2021-01-20T20:34:47+07:00,3637,114,227,51B23483,2021-01-20T20:34:47+07:00,4695.0,37.0,611.0,0


In [450]:
example.columns

Index(['RequestTime', 'StopId', 'RouteId', 'RouteVarId', 'BusId',
       'LocTimeStamp', 'Distance', 'Speed', 'TimeToStop', 'Unnamed'],
      dtype='object')

In [451]:
stop_pre.columns

Index(['RouteId', 'RouteVarId', 'StopId', 'Code', 'Name', 'StopType', 'Zone',
       'Ward', 'AddressNo', 'Street', 'SupportDisability', 'Status', 'Lng',
       'Lat', 'Search', 'Routes', 'RouteVarName', 'RouteVarShortName',
       'RouteNo', 'StartStop', 'EndStop', 'Distance', 'Outbound',
       'RunningTime', 'RouteName'],
      dtype='object')

In [456]:
example_full = example.merge(
    stop_pre,
    on=["StopId", "RouteId", "RouteVarId"],
    how="left",
    suffixes=("", "Dim")
)

In [467]:
example_full.drop_duplicates(["RouteId", "RouteVarId"])

,RequestTime,StopId,RouteId,RouteVarId,BusId,LocTimeStamp,Distance,Speed,TimeToStop,Unnamed,Code,Name,StopType,Zone,Ward,AddressNo,Street,SupportDisability,Status,Lng,Lat,Search,Routes,RouteVarName,RouteVarShortName,RouteNo,StartStop,EndStop,DistanceDim,Outbound,RunningTime,RouteName
0,2021-01-20T03:00:31+07:00,1389,114,228,51B23483,2021-01-20T03:00:31+07:00,7524.0,51.0,713.0,0,HHM 040,Công ty thoát nước đô thị,Trụ dừng,Huyện Hóc Môn,Xã Tân Thới Nhì,Đối diện Cty thoát nước đô thị,Quốc lộ 22,,Đang khai thác,106.567576,10.911485,Cttndt DdCtndt Ql22,"70-3, 74, 94",Lượt về,Bến xe An Sương,74,Bến xe Củ Chi,Bến xe An Sương,22440.0,False,45,Bến xe An Sương - Bến xe Củ Chi
202,2021-01-20T04:05:41+07:00,946,53,105,51B23483,2021-01-20T04:05:41+07:00,24256.0,29.0,4520.0,0,BX37,Bến xe Ngã tư Ga,Bến xe,Quận 12,None,Bến xe Ngã Tư Ga,Quốc lộ 1A,,Đang khai thác,106.678360,10.862157,BxNtG BxNTG Ql1A,"103, 32, 58, 59",Lượt đi: Bến xe Miền Tây - Bến xe Ngã tư Ga,Bến Xe Ngã Tư Ga,32,Bến xe Miền Tây,Bến xe Ngã tư Ga,25122.0,True,80,Bến Xe Miền Tây - Bến Xe Ngã Tư Ga
1014,2021-01-20T06:26:38+07:00,4737,53,106,51B23483,2021-01-20T06:26:38+07:00,3220.0,0.0,565.0,0,QGV 057,Trường THCS Lý Tự Trọng,Trụ dừng,Quận Gò Vấp,Phường 15,732,Lê Đức Thọ,Có,Đang khai thác,106.669393,10.847321,TTLTT 732 LDT,32,Lượt về: Bến xe Ngã tư Ga - Bến xe Miền Tây,Bến Xe Miền Tây,32,Bến xe Ngã tư Ga,Bến xe Miền Tây,25890.0,False,80,Bến Xe Miền Tây - Bến Xe Ngã Tư Ga
4850,2021-01-20T20:34:47+07:00,3637,114,227,51B23483,2021-01-20T20:34:47+07:00,4695.0,37.0,611.0,0,HHM 036,Cây xăng Hoàng Anh 2,Trụ dừng,Huyện Hóc Môn,Xã Tân Hiệp,428,Quốc lộ 22,,Đang khai thác,106.572071,10.905691,CxHA2 428 Ql22,74,Lượt đi,Bến xe Củ Chi,74,Bến xe An Sương,Bến xe Củ Chi,22529.0,True,45,Bến xe An Sương - Bến xe Củ Chi


In [490]:
example_full.groupby(["RouteId", "RouteVarId"]).LocTimeStamp.min().to_frame()

LocTimeStamp
RouteId RouteVarId                           
53      105         2021-01-20T04:05:41+07:00
        106         2021-01-20T06:26:38+07:00
114     227         2021-01-20T20:34:47+07:00
        228         2021-01-20T03:00:31+07:00

In [489]:
example_full.groupby(["RouteId", "RouteVarId"]).LocTimeStamp.max().to_frame()

LocTimeStamp
RouteId RouteVarId                           
53      105         2021-01-20T17:54:50+07:00
        106         2021-01-20T19:36:15+07:00
114     227         2021-01-20T21:01:19+07:00
        228         2021-01-20T03:25:32+07:00

In [509]:
stop_pre[stop_pre.RouteId==53].head()

,RouteId,RouteVarId,StopId,Code,Name,StopType,Zone,Ward,AddressNo,Street,SupportDisability,Status,Lng,Lat,Search,Routes,RouteVarName,RouteVarShortName,RouteNo,StartStop,EndStop,Distance,Outbound,RunningTime,RouteName
4336,53,105,725,BX 46,Bến xe Miền Tây,Bến xe,Quận Bình Tân,None,Bến xe Miền Tây,Kinh Dương Vương,,Đang khai thác,106.618317,10.740705,BxMT BxMT KDV,"10, 102, 139, 14, 148, 151, 32, 39, 41, 60-1, ...",Lượt đi: Bến xe Miền Tây - Bến xe Ngã tư Ga,Bến Xe Ngã Tư Ga,32,Bến xe Miền Tây,Bến xe Ngã tư Ga,25122.0,True,80,Bến Xe Miền Tây - Bến Xe Ngã Tư Ga
4337,53,105,140,QBT 010,Kinh Dương Vương,Trụ dừng,Quận Bình Tân,Phường An Lạc,7,Kinh Dương Vương,,Đang khai thác,106.623162,10.744919,KDV 7 KDV,"09, 139, 14, 148, 25, 32, 61, 62-1, 62-10, 62-...",Lượt đi: Bến xe Miền Tây - Bến xe Ngã tư Ga,Bến Xe Ngã Tư Ga,32,Bến xe Miền Tây,Bến xe Ngã tư Ga,25122.0,True,80,Bến Xe Miền Tây - Bến Xe Ngã Tư Ga
4338,53,105,718,Q6 048,Công viên Phú Lâm,Trụ dừng,Quận 6,None,209,Kinh Dương Vương,,Đang khai thác,106.626257,10.747549,CvPL 209 KDV,"09, 139, 14, 32, 62-1, 62-10, 62-2, 62-8, 84",Lượt đi: Bến xe Miền Tây - Bến xe Ngã tư Ga,Bến Xe Ngã Tư Ga,32,Bến xe Miền Tây,Bến xe Ngã tư Ga,25122.0,True,80,Bến Xe Miền Tây - Bến Xe Ngã Tư Ga
4339,53,105,715,Q6 049,Hồ Bơi Phú Lâm,Trụ dừng,Quận 6,None,157,Kinh Dương Vương,,Đang khai thác,106.627963,10.748914,HBPL 157 KDV,"09, 139, 14, 32, 62-1, 62-10, 62-2, 62-8, 84",Lượt đi: Bến xe Miền Tây - Bến xe Ngã tư Ga,Bến Xe Ngã Tư Ga,32,Bến xe Miền Tây,Bến xe Ngã tư Ga,25122.0,True,80,Bến Xe Miền Tây - Bến Xe Ngã Tư Ga
4340,53,105,720,Q6 050,Cư xá Phú Lâm,Trụ dừng,Quận 6,None,93,Kinh Dương Vương,,Đang khai thác,106.630914,10.751217,CxPL 93 KDV,"09, 139, 14, 32, 62-1, 62-10, 62-2, 62-8, 84",Lượt đi: Bến xe Miền Tây - Bến xe Ngã tư Ga,Bến Xe Ngã Tư Ga,32,Bến xe Miền Tây,Bến xe Ngã tư Ga,25122.0,True,80,Bến Xe Miền Tây - Bến Xe Ngã Tư Ga
